The code is running using NumPy.  The functions used are also available with torch and tensorflow, but 
if the Anaconda distribution of Python is installed, NumPy comes pre-installed and no further installation steps are necessary.

In the following lines, we refer to this algorithm as "EA"

In [ ]:
import numpy as np

First, the variables needed for the implementation are set. The time unit in the implementation is milliseconds (ms)

In [ ]:
# number of iterations in the Step_2
step_2_iterations = 4
# for every matrix containing samples in the way (t_Start, t_End) the columns are then numerated:
t_start_column = 0
t_end_column = 1
# Possible offset of the range (T_Start, t_0) for the creation of samples in step_1
offset = 10

Now, EA variables are set:
* In V_1 it is desirable to prioritize | t_End − E_i | over | t_Start− E_i |, accordingly,  b must be higher than a , thus b > a
* As V_2 corresponds with the main purpose of the test, it should be correspondingly higher than a and b , therefore c > b > a
* If E is empty, then V_1 = 0

In [ ]:
# T_Start: Instant when pre-crash recording time starts.
T_Start	= -5000
# t_0: Instant when pre-crash recording time ends.  
# As t_0 is the time when the recording algorithm start, it is normally set to zero.
t_0 = 0
# T: Pre-crash recording time established in the regulation.
T = (T_Start, 0)
# popsize: Population size in the Evolutive Algorithm.  Number of test samples created with the algorithm
popsize	= 4
# lambda_new: New generated samples in each iteration
lambda_new = 2
# The value obtained from the minimum is divided by 1000 so that the value of V_1 is comparable with the of V_2 (next criteria).
V_1_scale = 0.001

# For the fitness function:
a	= 1  # Factor to weight | t_Start − E_i [ t_Start ] | in V_1
b	= 3  # Factor to weight | t_End − E_i [ t_End ] | in V_1
c	= 5  # Factor to weight V_2 over V_1

# For the mutation function:
mu = 0  # Mean in normal distribution: for the standard normal distribution set in zero
sigma = 5 # Standard deviation in normal distribution: It is also known as the scale factor

error_hit_set
(Fehlertreff in Excel sheet) is the set with the samples, that have discovered an error within the test procedure.  At the beginning of the whole procedure the set is empty, but as the procedure discover errors in testing, it will be filled with them.
The next cell contains an example of a possible set of samples that had lead to an error.

In [ ]:
error_hit_set = np.array([[-4025, -300],[-3213, -1232]])

initialize_samples_array
Create set of random test samples in the range [T_start-offset , t_0+offset]

In [ ]:
def initialize_samples_array():
    # Generate random values for the t_start values in the range [0,1]
    t_start = np.random.rand(popsize, 1)
    
    # Generate random values for the t_End values in the range [0,t_start]
    t_End = np.random.rand(popsize, 1) * t_start
    
    # Concatenate the columns to create the array of samples in the way (t_start, t_End)
    array = np.concatenate((t_start, t_End), axis=1)
    
    # Establish the array in the range from T_start to t_0 plus the offset and round to have only integers 
    array = offset + np.rint(array * (T_Start - 2*offset))
    
    return array

In [10]:
def fitness_function(samples, error_hit_set):
    res_V_1 = V_1(samples, error_hit_set) + c*V_2(samples)

    return res_V_1

def V_1(samples_array, error_hit_set):
    # weight (t_Start, E_i[t_Starμ]) and t_End, E_i[t_End] in the samples and error_hit_set matrices 
    # with a and b correspondingly before the diffrence comparison between them 
    weights = np.array([[a,b]])
    samples_weigthed = samples_array * weights
    error_hit_set_weigthed = error_hit_set * weights
    
    # Calculate the absolute difference between each row in samples_weigthed and all rows in error_hit_set
    diff = np.abs(samples_weigthed[:, None, :] - error_hit_set_weigthed)

    # Sum the members for each sample case
    sum_evaluation = diff.sum(axis=2)
    
    # return the minimum for each case multiplied by V_1_scale
    return np.min(sum_evaluation, axis=1) * V_1_scale

def V_2(samples_array):
    # Get the t_end of samples
    t_end = samples_array[:, t_end_column]

    # If t_end>t_0 we return the diffrence, otherwise we return T_Start - t_end,
    # distance = np.where(t_end>t_0, t_end - t_0, T_Start - t_end)
    #
    # But as t_0 is set at zero then t_end - t_0 can be just t_end
    distance = np.where(t_end>t_0, t_end, T_Start - t_end)
    
    # We return only the positive numbers, as they are the ones not contained in the range [T_Start,t_0], 
    # othewise we return zero
    return distance.clip(min=0)